IMPORTANT LIBRARIES

In [25]:
import pefile
import re
import numpy as np
import pandas as pd
import os
import glob
from sklearn.preprocessing import LabelBinarizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import nltk
from sklearn.metrics import confusion_matrix
from nltk.tokenize import WhitespaceTokenizer
import string
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn import metrics
from sklearn.naive_bayes import MultinomialNB,GaussianNB,BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

[nltk_data] Downloading package punkt to

[nltk_data]     C:\Users\KIIT\AppData\Roaming\nltk_data...

[nltk_data]   Package punkt is already up-to-date!

[nltk_data] Downloading package stopwords to

[nltk_data]     C:\Users\KIIT\AppData\Roaming\nltk_data...

[nltk_data]   Package stopwords is already up-to-date!

[nltk_data] Downloading package wordnet to

[nltk_data]     C:\Users\KIIT\AppData\Roaming\nltk_data...

[nltk_data]   Package wordnet is already up-to-date!


DATA EXTRACTION

In [3]:
#Already extracted strings from each file using shell script extract_strings.sh 
# Now extracting dos header, nt header, file header, optional header and import functions from each dll file 
#then using extracted strings ,headers, functions to create a dataframe

In [4]:
#PATH---directory of applications/files
#PATH1---directory of strings files
#label---malware(0),benignware(1)
#get_headers_and_strings() returning list of dictionaries ,where each dictionary has headers and strings of particular file with label

def get_headers_and_strings(PATH,PATH1,label):
    lst = []
    for f in os.listdir(PATH):
        try: pe = pefile.PE(PATH+f)
        except pefile.PEFormatError: continue
        #return pe
        tmp_k = list(np.reshape(pe.DOS_HEADER.__keys__,-1))
        tmp_v = list(pe.DOS_HEADER.__unpacked_data_elms__)

        tmp_k.extend(np.reshape(pe.NT_HEADERS.__keys__,-1))
        tmp_v.extend(pe.NT_HEADERS.__unpacked_data_elms__)

        tmp_k.extend(np.reshape(pe.FILE_HEADER.__keys__,-1))
        tmp_v.extend(pe.FILE_HEADER.__unpacked_data_elms__)
        
        tmp_k.extend(np.reshape(pe.OPTIONAL_HEADER.__keys__,-1))
        tmp_v.extend(pe.OPTIONAL_HEADER.__unpacked_data_elms__)

        try:
            for entry in pe.DIRECTORY_ENTRY_IMPORT:
                tmp_k.append(entry.dll.decode('utf-8'))
                tmp_v.append(1)
                for fun in entry.imports:
                    tmp_k.append(fun.name.decode('utf-8'))
                    tmp_v.append(1)
        except : pass
        
        with open(PATH1+f.replace('.','')+".txt") as f1:
            tmp_str = ''
            for line in f1.read().splitlines() : tmp_str = tmp_str + line + ' '
        tmp_k.append("Strings")
        tmp_v.append(tmp_str.strip())
        
        tmp_k.append("label")
        tmp_v.append(label)

        lst.append(dict(zip(tmp_k,tmp_v)))
    return lst

In [5]:
#getting headers and strings
headers_b = get_headers_and_strings("C:\\Users\\KIIT\\Downloads\\IIT Palakkad\\Academics\\semester 2\\ai for cyber security\\assignment\\malware detection PE windows\\data\\data\\malware_dataset\\benignware\\",
                             "C:\\Users\\KIIT\\Downloads\\IIT Palakkad\\Academics\\semester 2\\ai for cyber security\\assignment\\malware detection PE windows\\ai cybersec-20220308T072953Z-001\\ai cybersec\\benignware_strings\\",1)
headers_m = get_headers_and_strings("C:\\Users\\KIIT\\Downloads\\IIT Palakkad\\Academics\\semester 2\\ai for cyber security\\assignment\\malware detection PE windows\\data\\data\\malware_dataset\\malware\\",
                             "C:\\Users\\KIIT\\Downloads\\IIT Palakkad\\Academics\\semester 2\\ai for cyber security\\assignment\\malware detection PE windows\\ai cybersec-20220308T072953Z-001\\ai cybersec\\malware_strings\\",0)

In [6]:
#creating dataframe
df = pd.DataFrame(headers_b)
df = df.append(headers_m)
df.reset_index(inplace=True,drop=True)
df.head(5)

,e_magic,e_cblp,e_cp,e_crlc,e_cparhdr,e_minalloc,e_maxalloc,e_ss,e_sp,e_csum,...,rtcRightCharVar,rtcR8ValFromBstr,kernel32.DLL,RtlMoveMemory,RtlZeroMemory,DirectInputCreateEx,SetTcpEntry,GetTcpTable,CopyMetaFileA,InitiateSystemShutdownA
0,23117,144,3,0,4,0,65535,0,184,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,23117,144,3,0,4,0,65535,0,184,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,23117,144,3,0,4,0,65535,0,184,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,23117,144,3,0,4,0,65535,0,184,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,23117,144,3,0,4,0,65535,0,184,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
df.shape

(819, 5621)

In [8]:
#saving dataframe
df.to_csv("df_pe.csv",index=False)

In [12]:
#fetching dataframe
df = pd.read_csv("df_pe.csv")
df.head(2)

,e_magic,e_cblp,e_cp,e_crlc,e_cparhdr,e_minalloc,e_maxalloc,e_ss,e_sp,e_csum,...,rtcRightCharVar,rtcR8ValFromBstr,kernel32.DLL,RtlMoveMemory,RtlZeroMemory,DirectInputCreateEx,SetTcpEntry,GetTcpTable,CopyMetaFileA,InitiateSystemShutdownA
0,23117,144,3,0,4,0,65535,0,184,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,23117,144,3,0,4,0,65535,0,184,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


GETTING DATAFRAME READY

In [13]:
#There are some features which are categorical ,so creating new features for each value of feature
#machine - no need as there is only 1 category given for all files
#Characteristics - created separate column for each value and new column value is 0 or 1 accordingly
#Signature - no need as there is only 1 category given for all files
#Subsystem - created separate column for each value and new column value is 0 or 1 accordingly
#DllCharacteristics - created separate column for each value and new column value is 0 or 1 accordingly

In [14]:
#function will drop categorical feature and create separate feature for each unique value of feature
def preprocess_cat_fea(df,feature):
    lb = LabelBinarizer()
    lb.fit(df[feature])
    trans = lb.transform(df[feature])
    trans_df = pd.DataFrame(trans,columns=[feature+"("+str(v)+")" for v in pd.unique(df[feature])],index=df.index)
    return pd.concat([df,trans_df],axis=1).drop([feature], axis=1)

In [15]:
df = preprocess_cat_fea(df,"Characteristics")
df = preprocess_cat_fea(df,"Subsystem")
df = preprocess_cat_fea(df,"DllCharacteristics")

In [16]:
df.shape

(819, 5660)

In [17]:
#dropping 2 columns as there values are same for all observations (reserver words)
df.drop(['e_res','e_res2'],axis=1,inplace=True)

In [18]:
df.shape

(819, 5658)

In [19]:
#there are some columns which have null values, so replacing them with 0 because this columns are of functions feature which are boolean
for c in df.columns:
    if(df[c].isnull().values.any()):print(c)

ntdll.dll

DbgPrint

vDbgPrintExWithPrefix

advapi32.dll

RegCloseKey

RegEnumKeyExW

RegOpenKeyW

RegQueryInfoKeyW

RegQueryValueExW

RegSetValueExW

advpack.dll

ExtractFilesW

RunSetupCommandW

kernel32.dll

CloseHandle

CopyFileW

CreateDirectoryW

CreateFileW

DeleteCriticalSection

DisableThreadLibraryCalls

EnterCriticalSection

EnumResourceNamesW

FindResourceW

FreeLibrary

GetCurrentProcess

GetCurrentProcessId

GetCurrentThreadId

GetDriveTypeW

GetEnvironmentVariableA

GetFileAttributesW

GetFileSizeEx

GetFileTime

GetLastError

GetModuleFileNameW

GetModuleHandleA

GetPrivateProfileStringW

GetProcAddress

GetProcessHeap

GetSystemTimeAsFileTime

GetTempFileNameW

GetTempPathW

GetTickCount

GetVersionExW

GlobalAlloc

GlobalLock

GlobalSize

GlobalUnlock

HeapAlloc

HeapFree

HeapReAlloc

InitializeCriticalSection

InterlockedCompareExchange

InterlockedDecrement

InterlockedExchange

InterlockedExchangeAdd

InterlockedIncrement

IsWow64Process

LeaveCriticalSection

Loa

In [20]:
df.fillna(0, inplace=True)

PREPROCESS STRINGS

In [21]:
#preprocess function
def preprocess(mod_data) : 
    #lowercase
    mod_data['Strings']= mod_data['Strings'].apply(lambda x: x.lower())
    #whitespace tokenizer
    tok = WhitespaceTokenizer()
    mod_data['Strings']= mod_data['Strings'].apply(lambda x: " ".join(tok.tokenize(x)))  
    return mod_data

In [22]:
#preprocessing strings
df = preprocess(df)

In [23]:
df.shape

(819, 5658)

COUNT VECTORIZER TO GET STRINGS VECTORS

In [26]:
#using Count Vectorizer to get vectors for each string
#then dropping original strings column and replacing it with new Count Vectorizer vectors
cv = CountVectorizer()
trans_df = pd.DataFrame(cv.fit_transform(df["Strings"]).toarray(),index=df.index)
df = pd.concat([df,trans_df],axis=1).drop(["Strings"], axis=1)
df.head(5)

,e_magic,e_cblp,e_cp,e_crlc,e_cparhdr,e_minalloc,e_maxalloc,e_ss,e_sp,e_csum,...,154722,154723,154724,154725,154726,154727,154728,154729,154730,154731
0,23117,144,3,0,4,0,65535,0,184,0,...,0,0,0,0,0,0,0,0,0,0
1,23117,144,3,0,4,0,65535,0,184,0,...,0,0,0,0,0,0,0,0,0,0
2,23117,144,3,0,4,0,65535,0,184,0,...,0,0,0,0,0,0,0,0,0,0
3,23117,144,3,0,4,0,65535,0,184,0,...,0,0,0,0,0,0,0,0,0,0
4,23117,144,3,0,4,0,65535,0,184,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
#splitting data into X(features) and Y(labels) for further operations
Y = df['label']
X = df.drop(['label'],axis=1)

FEATURE SCALING

In [28]:
#Feature scaling using MinMaxScaler, so that data will lie between (0,1)
mms = MinMaxScaler()
X = pd.DataFrame(mms.fit_transform(X),index=X.index)
X.head(5)

,0,1,2,3,4,5,6,7,8,9,...,160378,160379,160380,160381,160382,160383,160384,160385,160386,160387
0,0.0,0.01057,0.00022,0.0,1.0,0.0,1.0,0.0,0.554217,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.01057,0.00022,0.0,1.0,0.0,1.0,0.0,0.554217,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.01057,0.00022,0.0,1.0,0.0,1.0,0.0,0.554217,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.01057,0.00022,0.0,1.0,0.0,1.0,0.0,0.554217,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.01057,0.00022,0.0,1.0,0.0,1.0,0.0,0.554217,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


FEATURE SELECTIONS

In [33]:
#Removing features with 0 variance or same values
vt = VarianceThreshold(0.05)
X = pd.DataFrame(vt.fit_transform(X),index=X.index)
X.shape

(819, 751)

In [34]:
#selecting 10000 best feature after doing chi-squared on it
X = pd.DataFrame(SelectKBest(chi2, k=100).fit_transform(X, Y),index=X.index)
X

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0
1,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0
2,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0
3,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0
4,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
814,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
815,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
816,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.5
817,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


SPLITTING DATAFRAME IN TRAINING AND TESTING

In [35]:
#spliting Dataframe in training and testing set as 70% and 30%
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, shuffle=True, random_state=42)

In [36]:
#function for displaying accuracy,f1-score,confusion matrix,recall,precision
def show_report(test,pred,type):
  print(type+"=====================================================================================>>>\n")
  print("\nAccuracy : {:.4f}".format(accuracy_score(test, pred)))
  print("\nClassification report : \n", metrics.classification_report(test, pred))
  print("\nConfusion Matrix : \n", metrics.confusion_matrix(test, pred))

MULTINOMIAL NAIVE BAYES

In [37]:
#getting almost same accuracy on both training and testing dataset
mnb = MultinomialNB().fit(X_train,Y_train)
pred_train = mnb.predict(X_train)
show_report(Y_train,pred_train,"TRAIN")
pred_test = mnb.predict(X_test)
show_report(Y_test,pred_test,"TEST")

TRAIN=====================================================================================>>>





Accuracy : 0.8848



Classification report : 

               precision    recall  f1-score   support



           0       0.81      1.00      0.89       281

           1       1.00      0.78      0.87       292



    accuracy                           0.88       573

   macro avg       0.90      0.89      0.88       573

weighted avg       0.91      0.88      0.88       573





Confusion Matrix : 

 [[280   1]

 [ 65 227]]

TEST=====================================================================================>>>





Accuracy : 0.9024



Classification report : 

               precision    recall  f1-score   support



           0       0.84      0.99      0.91       118

           1       0.99      0.82      0.90       128



    accuracy                           0.90       246

   macro avg       0.91      0.91      0.90       246

weighted avg       0.92      0.90      0.90

GAUSSIAN NAIVE BAYES

In [38]:
#getting little better accuracy for training as compared to testing dataset
gnb = GaussianNB().fit(X_train,Y_train)
pred_train = gnb.predict(X_train)
show_report(Y_train,pred_train,"TRAIN")
pred_test = gnb.predict(X_test)
show_report(Y_test,pred_test,"TEST")

TRAIN=====================================================================================>>>





Accuracy : 0.8482



Classification report : 

               precision    recall  f1-score   support



           0       0.76      1.00      0.87       281

           1       1.00      0.70      0.82       292



    accuracy                           0.85       573

   macro avg       0.88      0.85      0.85       573

weighted avg       0.88      0.85      0.85       573





Confusion Matrix : 

 [[281   0]

 [ 87 205]]

TEST=====================================================================================>>>





Accuracy : 0.8618



Classification report : 

               precision    recall  f1-score   support



           0       0.78      1.00      0.87       118

           1       1.00      0.73      0.85       128



    accuracy                           0.86       246

   macro avg       0.89      0.87      0.86       246

weighted avg       0.89      0.86      0.86

BERNOULLI NAIVE BAYES

In [39]:
#getting average accuracy on both training and testing dataset
bnb = BernoulliNB().fit(X_train,Y_train)
pred_train = bnb.predict(X_train)
show_report(Y_train,pred_train,"TRAIN")
pred_test = bnb.predict(X_test)
show_report(Y_test,pred_test,"TEST")

TRAIN=====================================================================================>>>





Accuracy : 0.8394



Classification report : 

               precision    recall  f1-score   support



           0       0.77      0.96      0.85       281

           1       0.95      0.72      0.82       292



    accuracy                           0.84       573

   macro avg       0.86      0.84      0.84       573

weighted avg       0.86      0.84      0.84       573





Confusion Matrix : 

 [[270  11]

 [ 81 211]]

TEST=====================================================================================>>>





Accuracy : 0.8496



Classification report : 

               precision    recall  f1-score   support



           0       0.78      0.95      0.86       118

           1       0.94      0.76      0.84       128



    accuracy                           0.85       246

   macro avg       0.86      0.85      0.85       246

weighted avg       0.87      0.85      0.85

LOGISTIC REGRESSION

In [40]:
#getting very good accuracy on both training and testing dataset
lr = LogisticRegression(max_iter=1000,n_jobs=-1).fit(X_train,Y_train)
pred_train = lr.predict(X_train)
show_report(Y_train,pred_train,"TRAIN")
pred_test = lr.predict(X_test)
show_report(Y_test,pred_test,"TEST")

TRAIN=====================================================================================>>>





Accuracy : 0.9791



Classification report : 

               precision    recall  f1-score   support



           0       0.97      0.99      0.98       281

           1       0.99      0.97      0.98       292



    accuracy                           0.98       573

   macro avg       0.98      0.98      0.98       573

weighted avg       0.98      0.98      0.98       573





Confusion Matrix : 

 [[278   3]

 [  9 283]]

TEST=====================================================================================>>>





Accuracy : 0.9919



Classification report : 

               precision    recall  f1-score   support



           0       0.99      0.99      0.99       118

           1       0.99      0.99      0.99       128



    accuracy                           0.99       246

   macro avg       0.99      0.99      0.99       246

weighted avg       0.99      0.99      0.99

RANDOM FOREST

In [41]:
#getting very high accuracy for training and getting very good but not as much high accuracy on testing dataset as compared to training dataset
#show overfitting of the model
rf = RandomForestClassifier(n_estimators=200, max_depth=200, n_jobs=-1, random_state=0).fit(X_train,Y_train)
pred_train = rf.predict(X_train)
show_report(Y_train,pred_train,"TRAIN")
pred_test = rf.predict(X_test)
show_report(Y_test,pred_test,"TEST")

TRAIN=====================================================================================>>>





Accuracy : 0.9948



Classification report : 

               precision    recall  f1-score   support



           0       0.99      1.00      0.99       281

           1       1.00      0.99      0.99       292



    accuracy                           0.99       573

   macro avg       0.99      0.99      0.99       573

weighted avg       0.99      0.99      0.99       573





Confusion Matrix : 

 [[280   1]

 [  2 290]]

TEST=====================================================================================>>>





Accuracy : 0.9919



Classification report : 

               precision    recall  f1-score   support



           0       1.00      0.98      0.99       118

           1       0.98      1.00      0.99       128



    accuracy                           0.99       246

   macro avg       0.99      0.99      0.99       246

weighted avg       0.99      0.99      0.99

SUPPORT VECTOR MACHINE

In [42]:
#getting good accuracy on both training and testing dataset
#show overfitting of model
svc = SVC(C=0.4).fit(X_train,Y_train)
pred_train = svc.predict(X_train)
show_report(Y_train,pred_train,"TRAIN")
pred_test = svc.predict(X_test)
show_report(Y_test,pred_test,"TEST")

TRAIN=====================================================================================>>>





Accuracy : 0.9703



Classification report : 

               precision    recall  f1-score   support



           0       0.97      0.96      0.97       281

           1       0.97      0.98      0.97       292



    accuracy                           0.97       573

   macro avg       0.97      0.97      0.97       573

weighted avg       0.97      0.97      0.97       573





Confusion Matrix : 

 [[271  10]

 [  7 285]]

TEST=====================================================================================>>>





Accuracy : 0.9797



Classification report : 

               precision    recall  f1-score   support



           0       0.99      0.97      0.98       118

           1       0.97      0.99      0.98       128



    accuracy                           0.98       246

   macro avg       0.98      0.98      0.98       246

weighted avg       0.98      0.98      0.98

BEST MODEL (ACCURACY)

BernoulliNB(0.8496) < MultinomialNB(0.9024) < GaussianNB(0.8618) < SVM-RBF(0.9797) < Random Forest(0.9919) = Logistic Regression(0.9719) 